# 팀 전체 전처리 코드

### 전처리 한 부분 설명
1. brandnm 결측치는 전부 TP : Mixed로 치환
2. app_yn 결측치는 모두 비유저 : N 으로 치환
3. reg_date 결측치는 5개 : drop처리
4. 파생변수 days는 각 요일을 의미 (김아람님 컬럼명 order_day에 해당) / str타입
5. 파생변수 day_type는 주중, 주말 구분하는 컬럼 / str타입
6. 파생변수 month는 각 결제일의 월을 의미 / str타입
7. 파생변수 How_long은 가입 후 결제일까지 걸린 시간 (안태윤님 컬럼명 trans_period에 해당) / int타입

In [27]:
# 필요한 패키지는 추가로 설치하시면 됩니다!! 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings('ignore')

In [28]:
## 시각화 폰트 설정 함수입니다.

def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux
        # colab에서는 runtime을 <꼭> 재시작 해야함.
        # 런타임을 재시작 하지 않고 폰트 설치를 하면 기본 설정 폰트가 로드되어 한글이 깨짐.
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null

        import matplotlib.font_manager as fm

        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

In [29]:
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당.
a = get_font_family()
# 폰트설정
import matplotlib.pyplot as plt 
plt.rc("font", family = a)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# ggplot으로 그래프 스타일 설정
#plt.style.use("ggplot")

In [30]:
# 데이터 불러오기
df = pd.read_excel('./data/클로젯셰어_과제DB 대체 자료.xlsx')

In [31]:
# brandnm 결측치 보완
df['brandnm'].fillna("Mixed", inplace=True) # TP는 전부 결측치 ==> Mixed로 수정

# app_yn 결측치 보완
df['app_yn'].fillna("N", inplace=True) # 이용하지 않은 사람 ==> N으로 처리

# 결측치 제거 ==> reg_data 누락 5개 삭제처리
df = df.dropna(axis=0)

# 인덱스 재설정 
df = df.reset_index().drop("index", axis=1)

In [32]:
# 요일 표시 (0~6)
df['days'] = df.order_date.dt.weekday  # 주문 요일 넣어주기 (월=0)
df

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn,days
0,Gucci,flower dionysus shoulder bag beige,a161237,2019-12-06,2021-01-15,7days,BA,Y,4
1,El estilo de Claire,beads cropped jacket black,a1140859,2021-01-02,2021-01-15,4days,O,N,4
2,Danha,embroidery point oriental mini skirt blue,a1140859,2021-01-02,2021-01-15,4days,B,N,4
3,Danha,graphics pattern top pink,a1140859,2021-01-02,2021-01-15,4days,T,N,4
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,2021-01-01,2021-01-15,7days,O,Y,4
...,...,...,...,...,...,...,...,...,...
1461,Yves Saint Laurent,logo point clutch bag pink,a119721,2019-04-10,2021-03-01,7days,BA,Y,0
1462,Prada,saffiano lux M gray,a1146069,2021-02-11,2021-03-01,7days,BA,Y,0
1463,Burberry,basic pattern sweater beige,a1147848,2021-02-28,2021-03-01,7days,T,Y,0
1464,Chanel,graphics pattern sweat-shirts white,a1147788,2021-02-28,2021-03-01,7days,T,Y,0


In [33]:
# 공휴일 표시 (공휴일==7)
holiday_list = ['2021-01-01', '2021-02-11', '2021-02-12', '2021-02-13', '2021-03-01']

for i in range(len(df)) :
    if df['order_date'].loc[i].strftime('%Y-%m-%d') in holiday_list :
        df['days'].loc[i] = 7

In [34]:
# 주중 주말,공휴일 구분 ==> 주중과 휴일 2가지로 구분 (object)

df['day_type'] = "주중"

# day_type 주중 : 0 / 주말 : 1
for num in range(len(df)) :
    if df['days'].iloc[num] > 4 :
        df['day_type'].iloc[num] = "휴일"

In [35]:
# 요일 한글로 바꾸자
for day_num in range(len(df)) :
    a = df['days'].iloc[day_num]
    days=['월요일','화요일','수요일','목요일','금요일','토요일','일요일','공휴일']
    df['days'].iloc[day_num] = days[a]

In [36]:
# 월단위 만들기 ==> object
df['month'] = df.order_date.dt.strftime('%m')
display(df.head())

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn,days,day_type,month
0,Gucci,flower dionysus shoulder bag beige,a161237,2019-12-06,2021-01-15,7days,BA,Y,금요일,주중,01
1,El estilo de Claire,beads cropped jacket black,a1140859,2021-01-02,2021-01-15,4days,O,N,금요일,주중,01
2,Danha,embroidery point oriental mini skirt blue,a1140859,2021-01-02,2021-01-15,4days,B,N,금요일,주중,01
3,Danha,graphics pattern top pink,a1140859,2021-01-02,2021-01-15,4days,T,N,금요일,주중,01
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,2021-01-01,2021-01-15,7days,O,Y,금요일,주중,01


In [37]:
# 이용일까지 걸린 시간 컬럼 만들기
df['How_long'] = df['order_date'] - df['reg_date']

# timedelta가 아닌 int 형으로 바꾸기 
df['How_long'] = df['How_long'].dt.days

In [38]:
df

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn,days,day_type,month,How_long
0,Gucci,flower dionysus shoulder bag beige,a161237,2019-12-06,2021-01-15,7days,BA,Y,금요일,주중,01,406
1,El estilo de Claire,beads cropped jacket black,a1140859,2021-01-02,2021-01-15,4days,O,N,금요일,주중,01,13
2,Danha,embroidery point oriental mini skirt blue,a1140859,2021-01-02,2021-01-15,4days,B,N,금요일,주중,01,13
3,Danha,graphics pattern top pink,a1140859,2021-01-02,2021-01-15,4days,T,N,금요일,주중,01,13
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,2021-01-01,2021-01-15,7days,O,Y,금요일,주중,01,14
...,...,...,...,...,...,...,...,...,...,...,...,...
1461,Yves Saint Laurent,logo point clutch bag pink,a119721,2019-04-10,2021-03-01,7days,BA,Y,공휴일,휴일,03,691
1462,Prada,saffiano lux M gray,a1146069,2021-02-11,2021-03-01,7days,BA,Y,공휴일,휴일,03,18
1463,Burberry,basic pattern sweater beige,a1147848,2021-02-28,2021-03-01,7days,T,Y,공휴일,휴일,03,1
1464,Chanel,graphics pattern sweat-shirts white,a1147788,2021-02-28,2021-03-01,7days,T,Y,공휴일,휴일,03,1


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466 entries, 0 to 1465
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   brandnm     1466 non-null   object        
 1   goodsnm     1466 non-null   object        
 2   USER        1466 non-null   object        
 3   reg_date    1466 non-null   datetime64[ns]
 4   order_date  1466 non-null   datetime64[ns]
 5   p_type      1466 non-null   object        
 6   sub_type    1466 non-null   object        
 7   app_yn      1466 non-null   object        
 8   days        1466 non-null   object        
 9   day_type    1466 non-null   object        
 10  month       1466 non-null   object        
 11  How_long    1466 non-null   int64         
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 137.6+ KB


In [40]:
df.to_csv("./data/팀_basline_csv.csv")